# Implementation of a CNN Fire module for SqueezeNet
*by Marvin Bertin*
<img src="../../images/keras-tensorflow-logo.jpg" width="400">

# Imports

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

In [7]:
layers =  tf.contrib.keras.layers
models = tf.contrib.keras.models

# Implementation of Fire Module

<img src="../../images/fire_module.png" width="600">

In [3]:
def fire_module(x, fire_id, squeeze=16, expand=64):
    sq1x1 = "squeeze1x1"
    exp1x1 = "expand1x1"
    exp3x3 = "expand3x3"
    relu = "relu_"
    s_id = 'fire' + str(fire_id) + '/'

    # Squeeze layer
    x = layers.Convolution2D(squeeze, (1,1), padding='valid', name=s_id + sq1x1)(x)
    x = layers.Activation('relu', name=s_id + relu + sq1x1)

    # Expand layer 1x1 filters
    left = layers.Convolution2D(expand, (1,1), padding='valid', name=s_id + exp1x1)(x)
    left = layers.Activation('relu', name=s_id + relu + exp1x1)(left)

    # Expand layer 3x3 filters
    right = layers.Convolution2D(expand, (3,3), padding='same', name=s_id + exp3x3)(x)
    right = layers.Activation('relu', name=s_id + relu + exp3x3)(right)

    # concatenate outputs
    x = layers.concatenate([left, right], axis=3, name=s_id + 'concat')

    
    return x

# Implementation of SqueezeNet

In [4]:
def SqueezeNet(input_shape=(32,32,3), classes=10):
    
    img_input = layers.Input(shape=input_shape)
    
    x = layers.Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = layers.Activation('relu', name='relu_conv1')(x)
#     x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = layers.Dropout(0.5, name='drop9')(x)

    x = layers.Convolution2D(classes, (1, 1), padding='valid', name='conv10')(x)
    x = layers.Activation('relu', name='relu_conv10')(x)
    x = layers.GlobalAveragePooling2D()(x)
    out = layers.Activation('softmax', name='loss')(x)

    model = models.Model(img_input, out, name='squeezenet')

    return model

# Inspect SqueezeNet Architecture

In [9]:
squeeze_net = SqueezeNet()
squeeze_net.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 15, 15, 64)    1792        input_3[0][0]                    
____________________________________________________________________________________________________
relu_conv1 (Activation)          (None, 15, 15, 64)    0           conv1[0][0]                      
____________________________________________________________________________________________________
fire2/squeeze1x1 (Conv2D)        (None, 15, 15, 16)    1040        relu_conv1[0][0]                 
___________________________________________________________________________________________

## Next Lesson
### Train and Evaluate SqueezeNet
- Image classification task with Cifar10

<img src="../../images/divider.png" width="100">